In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import json
import re
import requests
from requests.auth import HTTPBasicAuth
from datetime import date
from datetime import datetime
from dateutil.relativedelta import relativedelta
from sqlalchemy import create_engine
from multiprocessing.dummy import Pool as ThreadPool
import urllib

# import ssdeep
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
#get url's content from solr

def get_content(url):
    df = pd.DataFrame()
    response = requests.get(url,timeout=5)
    data = json.loads(response.text)
    cursor = '*'
    i=0
    while (len(data['response']['docs'])==100):
        i+=1
        response = requests.get(url,timeout=5)
        data = json.loads(response.text)
        cursor = data['nextCursorMark']
        try:
            temp = pd.DataFrame.from_dict(data['response']['docs'])
            df = pd.concat([df,temp],ignore_index=True)
        except Exception:
            pass
    print(i,len(temp))
    if len(df)>0:
        return df
df=get_content()

138 93


In [3]:
#creating tree structure for all url's based on the child and parent url's\
#   to find the leaf nodes

class Node:
    def __init__(self, val, count):
        self.val = val
        self.count = count
        self.children = dict()
        
        
class Tree:
    import re
    def __init__(self, l):
        self.root = Node("", 0)
        self._build(l)

    def _build(self, l):

        for row in l:
#             print(row)
#             path_string = re.search(r"en_us/(.*).html(.*)", row)
            path_string=re.search(r"(.*).html(.*)",row)
#             print(path_string)
            if path_string:
                path = path_string.group(1).split('/')
#                 print(path)
                current_node = self.root
                for node in path:
                    if node in current_node.children:
                        current_node = current_node.children[node]
                    else:
                        current_node.children[node] = Node(node, 1)
                        current_node = current_node.children[node]
                        

    def getLeaf(self, showCount=False):
        """
        return all leaf sites

        showCount: Boolean, if set to True, the list returned will contain the number of times leaf sites are visited
        """

        def _helper(leaf, path, node, showCount=False):
#             print(node.children)
            if not node:
                return
            if len(node.children) == 0:
                if showCount:
                    leaf.append([path+'/'+node.val, node.count])
                    
                else:
#                     print(node.val)
                    leaf.append([path+'/'+node.val])
                return
            else:
                for child in node.children:
                    _helper(leaf, path+'/'+node.val, node.children[child], showCount)

        leaf = []
#         transition={}
        _helper(leaf, "", self.root, showCount)

        return (leaf)
    
# path_all=['https://one.walmart.com'+i for i in list(df['url'].head(10))]
test=Tree(list(df['url']))

leaf_nodes=test.getLeaf()

leaves=['/'+i[0][1:]+'.html' for i in leaf_nodes]

In [4]:
def remove_duplicates(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

In [5]:
def web_parser(url):
#     print(type(url))
        username = 'username'
        password = 'password'

        response=requests.get(url, auth=HTTPBasicAuth(username, password))
        if response.status_code != 200:
            response=requests.get(url, auth=HTTPBasicAuth(username, password))
            text=response.text.encode('utf-8')
            soup = BeautifulSoup(text,'html.parser')
            
        try:    
            response1=requests.get(url.split('.html')[0]+'.children.json', auth=HTTPBasicAuth(username, password))
            response_list=json.loads(response1.text.encode('utf-8'))
            for i in response_list:
                if(i.get('cq:lastModified')!=None):
                    last_modified=i['cq:lastModified']
                    return last_modified,soup
            return soup
        except Exception as e:
            return soup
        else:
            return 'not a valid format'

In [6]:
#parse html to get text data

def subtitle1(soup):
    data={'text':'','subtitles':''}
    classes = []
    subtitle_classes=[]
    text_classes=[]
    header_classes=[]
    footer_classes=[]
    hidden_classes=[]
    sub=[]
    for element in soup.find_all(class_=True):
        classes.extend(element["class"])
    classes=remove_duplicates(classes)
    for item in classes:
        if item.find("title")!=-1:
            if "header" not in item:
#                 print(item)
                subtitle_classes.append(item)
    for item in classes:
        if item.find("text")!=-1:
                text_classes.append(item)
    for item in classes:
        if item.find("header")!=-1:
                header_classes.append(item)
    for item in classes:
        if item.find("footer")!=-1:
                text_classes.append(item)
    for item in classes:
        if item.find("hidden")!=-1:
                hidden_classes.append(item)
    
#killing unneccessary headers, footer, style components etc
    for script in soup(["title","script", "style","label",'h',"head"]): #delete the title as well
    #     print(script)
        script.decompose()    # rip it out
    for item in header_classes:
        for i in soup.findAll(class_=item):
            i.decompose()
    for item in footer_classes:
        for i in soup.findAll(class_=item):
            i.decompose()
    for item in hidden_classes:
        for i in soup.findAll(class_=item):
            i.decompose()
    for item in classes:
        if item.find("not-to-translate")!=-1:
            for i in soup.findAll(class_=item):
                i.decompose()
        
    
#     complete_text = soup.get_text()
# #     print(text)
#     # break into lines and remove leading and trailing space on each
#     lines = (line.strip() for line in complete_text.splitlines())
#     # # break multi-headlines into a line each
#     chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
#     # # drop blank lines
#     complete_text = ' '.join(chunk.replace('\n','').replace(u'\xa0',' ').replace('\t',' ') for chunk in chunks if chunk)

#find subtitles in the pages
    for ite in subtitle_classes:
        for i in soup.find_all(class_=ite):
            if("notifications" not in ite):
                a=i.get_text().replace('\n',' ').replace('\t',' ')
                a=re.sub(' +',' ',a).encode('ascii', 'ignore').decode("utf-8")
#                 re.sub(r"[^a-zA-Z0-9\s./]+"," ",
                if a!=' ':
                    sub.append(a)
                i.decompose()
        for i in soup.find_all(re.compile(r'h[123]')):
            a=i.get_text().replace('\n',' ').replace('\t',' ')
            a=re.sub(' +',' ',a).encode('ascii', 'ignore').decode("utf-8")
            if a!=' ':
                sub.append(a)
            i.decompose()
#         for i in soup.find_all(class_="white-text"):
#             print(i.get_text())
#             a=i.get_text().replace('\n',' ').replace(u'\xa0',' ').replace('\t',' ')
#             sub.append(a)
#             i.decompose()
    
#     sub=remove_duplicates(sub)
    sub=','.join(list(filter(None, sub)))

    
#extract text content of the pages
    t=[]
    for ite in text_classes:
        for i in soup.findAll(class_=ite):
            if("notifications" not in ite):
                a=i.get_text().replace('\n',' ').replace('\t',' ')
                if a!=' ':
                    t.append(a)
                i.decompose()
        for i in soup.find_all(re.compile(r'h[123]')):
            a=i.get_text().replace('\n',' ').replace('\t',' ')
            if a!=' ':
                t.append(a)
            i.decompose()

    for i in soup("p"):
#         print('hello')
#         print("p::",i.get_text())
        a=i.get_text().replace('\n',' ').replace('\t',' ')
#         a=a.replace('   ','')
        t.append(a)
        i.decompose()
   
    for i in soup("div", class_="tile parbase section"):
#         print("tile parbase:::",i.get_text())
        a=i.get_text().replace('\n',' ').replace('\t',' ')
#         a=a.replace('   ','')
        t.append(a)
        i.decompose()
  
# print(6)
        

    for i in soup("div",class_='caption image-width caption-alignment center imagefont-regular caption-above'):
#         print("caption image:::",i.get_text())
        a=i.get_text().replace('\n',' ').replace(u'\xa0',' ').replace('\t',' ')
#         a=a.replace('   ','')
        t.append(a)
#         print(7)
        i.decompose()
        
#     print("l",t)
#     print(t)
    t = ' '.join(list(filter(None, t)))
#     print(t)
    
        
    for i in soup.find_all(re.compile(r'h\d+')):
        #         print(i)

        try:
            if i.find_parent().attrs['class'] != ['dv-item-info']:
                #                 print('i decomposed:{}'.format(i))
                i.decompose()
        except:
            pass
#     print(soup.get_text())   

# # kill all script and style elements
    
    
    #kill header    
    for script in soup("div", class_="hide-on-print"):
        script.decompose()

    #kill header    
    for script in soup("div", class_="article-tmpl-full-width section"):
        script.decompose()

    #kill the directory tab
    for script in soup("div", class_="breadcrumb section"):
        script.decompose()

    #kill the navegation tab 
    for script in soup("div", class_="in-page-navigation ddl align-center"):
        script.decompose()       

    for script in soup("div",id="knowledgecheck",class_='in-page-navigation-sections'):
        script.decompose()


    for script in soup("div", class_="navigation-section-title"):
        script.decompose()    

   
    for script in soup("div",class_="output_wrapper"):
        script.decompose()

    for script in soup("div",class_='html-component section'):
        script.decompose()

    for script in soup("div",id="acs-commons-env-indicator"):
        script.decompose()

    for script in soup("div",class_="margin-bottom-0 table-component adp-richtext-circle"):
        script.decompose()


        
     #get text
  
    text = soup.get_text()
#     print(text)
    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # # drop blank lines
    text = ' '.join(chunk.replace('\n','').replace('\t',' ') for chunk in chunks if chunk)
    data['text'] = (re.sub(' +', ' ',t + " " + text)).encode('ascii', 'ignore').decode("utf-8")
# re.sub(r"[^a-zA-Z0-9,\s./]+", " ",
#     data['text']=(re.sub(' +', ' ',re.sub(r"[^a-zA-Z0-9\s./']+"," ",t+" "+text)))
    data['subtitles']=(sub)
#     data['complete_text']=(re.sub(' +', ' ',re.sub(r"[^a-zA-Z0-9\s./]+"," ",complete_text)))
    
    return data

In [7]:
# subtitle1(soup)
stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]


In [13]:
#workflow for parsing a web url and getting text content and subtitles out of it
broken=[]
data=[]
data_updated=[]
failed_urls=[]
count=0
today = datetime.today()


def myfn(url):
#     conn = connector.get_connector(name = 'AAL_cognitive_service_dev')
    try:
        i=url[1]
#         modified_date=url['last_modified'].to_numpy()[0]
        modified_date=url[0]
        try:
            date_modified,soup=web_parser(i)
        except:
            soup=web_parser(i)
#         print(i)
#         print(soup)
        if type(soup) is str:
            print(soup)
            return 
        if '404' in soup('title')[0].get_text():
            broken.append(i)
        else:
            try:
                title=soup.title.string
            except:
                title=''
            try:
                desc=soup.find('meta', attrs={'name': 'description'}).attrs['content']
            except:
                desc=''
            page_data=subtitle1(soup)
            page_source="None"
            if i[i.find('/content/'):].split('/')[0] is not None:
                page_source=i[i.find('/content/'):].split('/')[2]
                
#             print(desc)
        
#             print(soup)
            leaf_node=0
#             if i in leaves:
#                 leaf_node=1
#             try:
#                 publish_date=soup.find('meta', attrs={'name': 'publishDate'}).attrs['content']
#             except:
#                 publish_date=''
#             print(publish_date)
            if(date_modified):
                try:
                    difference=(datetime.today()-datetime.strptime(date_modified, "%Y-%m-%dT%H:%M:%S.%fZ" )).days
                except:
                    difference=(datetime.today()-datetime.strptime(date_modified, "%Y-%m-%dT%H:%M:%SZ" )).days
            else:
                try:
                    difference=(datetime.today()-datetime.strptime(modified_date, "%Y-%m-%dT%H:%M:%S.%fZ" )).days
                except:
                    difference=(datetime.today()-datetime.strptime(modified_date, "%Y-%m-%dT%H:%M:%SZ" )).days
            
#             if difference<2:
#                 env="dev"
#                 db_name = 'cognitive-service-'+env
#                 conn = engine.connect()
#                 query="select * from  where content_url=\'"+i+"\'"
#                 result=conn.execute(query)
#                 re=result.fetchall()
# #                 print(re)
#                 if len(re)!=0:
#                     temp=re[0][2]+re[0][3].replace(","," ")
#                     comp=page_data['text']+page_data['subtitles'].replace(","," ")
#                     similarity=(cosine_sim(temp,comp))
                    
#                     print(similarity)
#                     if(similarity<0.9):
#                         content_updated=1
#                         row_updated={'content_url':i,'page_title':title,'page_text':page_data['text'],'page_sub_titles':page_data['subtitles'],'page_desc':desc,'leaf_node':1,'page_source':page_source,'parse_date':today,"content_updated_recently":content_updated}
#                         data_updated.append(row_updated)
#                     else:
#                         content_updated=0
#                 else:
#                     content_updated=0
#                 return re
#             else:
#                 content_updated=0
            
                
            
            row={'content_url':i,'page_title':title,'page_text':page_data['text'],'page_sub_titles':page_data['subtitles'],'page_desc':desc,'leaf_node':1,'page_source':page_source,'parse_date':today,"content_updated_recently":content_updated}
#             print(row)
#            pd.DataFrame(row).to_sql(con = conn, name = 'LCS_content_parsing', if_exists = 'append')
#             print(row)
            data.append(row)
    except Exception as e:
        print(e)
#         print(i)
        failed_urls.append(i)

In [ ]:
#implement 3500 threads to run content extraction for 3000 urls
pool=ThreadPool(2000)
pool.map(myfn,df.head(10000).to_numpy())
pool.close()
pool.join()

not a valid htmlnot a valid html
not a valid html

not a valid html
HTTPSConnectionPool(host='electronicjournal.prod.walmart.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:777)'),))


In [ ]:
# #extracted data to dataframe for easy access
data=pd.DataFrame(data)
data_updated=pd.DataFrame(data_updated)

In [11]:
print(df.shape)
print(data.shape)
print(data_updated.shape)

(13796, 2)
(95, 9)
(0, 0)


In [12]:
failed_urls

[]

In [ ]:
df_data=pd.read_excel('content_extraction_updated.xlsx')
df_data=pd.concat([df_data,data_updated])
df_data
days=datetime.today() - relativedelta( days = +10 )
# days
df_data_ten_days = df_data[df_data['parse_date'] >= days]
df_data_ten_days

In [ ]:
data_updated.to_excel('content_extraction_updated.xlsx', index=False)

In [ ]:
#delete data from database
def db_execute(query):
    cxn = engine.connect()
    trans = cxn.begin()
    cxn.execute(query)
    trans.commit()
    cxn.close()
db_execute('delete from [dbo].[LCS_content_parsing] where 1=1')

In [26]:
#create connection to dev mysql server
env="dev"
db_name = 'cognitive-service-'+env

engine = create_engine()
conn = engine.connect()

In [28]:
#batch insertion of data into mysql server
data.to_sql(con = conn, name = 'table_name', index = False, if_exists = 'append')
conn.close()